[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/itmorn/AI.handbook/blob/main/DL/torch/nn/Normalization/LayerNorm.ipynb)

# LayerNorm
在4D输入(NCHW)上应用Layer Normalization，论文参考[Layer Normalization](https://arxiv.org/abs/1607.06450)

**定义**：  
torch.nn.LayerNorm(normalized_shape, eps=1e-05, elementwise_affine=True, device=None, dtype=None)

**公式**：  
$$y = \frac{x - \mathrm{E}[x]}{ \sqrt{\mathrm{Var}[x] + \epsilon}} * \gamma + \beta$$

和BN类似，参数可参考BN的讲解

# 图解train模式下的前向传播过程

<p align="center">
<a href="https://raw.githubusercontent.com/itmorn/AI.handbook/main/DL/torch/nn/Normalization/imgs/LayerNorm.svg">
<img src="./imgs/LayerNorm.svg"
    width="2000" /></a></p>


In [1]:
# 调包计算
import torch
import torch.nn as nn

input1 = torch.tensor([
    [
        [[1, 6],
         [9, 4]],
        [[12, 18],
         [13, 11]]],
    [
        [[2, 7],
         [3, 8]],
        [[19, 17],
         [15, 11]]
    ]
], dtype=torch.float32)
print("input1:\n", input1,"\n")

N, C, H, W = 2, 2, 2, 2

m = nn.LayerNorm(normalized_shape=[C, H, W], eps=1e-05, elementwise_affine=True)

output = m(input1)
print("output:\n", output,"\n")

input1:
 tensor([[[[ 1.,  6.],
          [ 9.,  4.]],

         [[12., 18.],
          [13., 11.]]],


        [[[ 2.,  7.],
          [ 3.,  8.]],

         [[19., 17.],
          [15., 11.]]]]) 

output:
 tensor([[[[-1.6199, -0.6381],
          [-0.0491, -1.0308]],

         [[ 0.5400,  1.7181],
          [ 0.7363,  0.3436]]],


        [[[-1.3908, -0.5479],
          [-1.2222, -0.3793]],

         [[ 1.4751,  1.1379],
          [ 0.8008,  0.1264]]]], grad_fn=<NativeLayerNormBackward0>) 



In [8]:
# 手工计算
import torch

# 这里涉及到组的计算，编程实现比较麻烦，展示起来不直观，这里就手工计算第1组，证明正确性即可。
input1 = torch.tensor([
    [
        [[1, 6],
         [9, 4]],
        [[12, 18],
         [13, 11]]],
    [
        [[2, 7],
         [3, 8]],
        [[19, 17],
         [15, 11]]
    ]
], dtype=torch.float32)

print("input1:\n", input1, "\n")

# 第1步：求均值和方差：
VarX, EX = torch.var_mean(input1, dim=(1,2,3),unbiased=False)  # NCHW
print("Ex:\n", EX, "\n")
print("VarX:\n", VarX, "\n")

# 第2步：减去均值：
result2 = input1-EX
print("input1-Ex:\n", result2, "\n")

# 第3步：求sqrt(VarX+eps)：
eps = 1e-5
result3 = torch.sqrt(VarX+eps)
print("sqrt(VarX+eps):\n", result3, "\n")

# 第4步：第2步的结果/第3步的结果，完成batch内的数据规范化:
result4 = result2/result3
print("(input1-Ex)/sqrt(VarX+eps):\n", result4, "\n")

# 第5步：使用γ=1，β=0 进行再校正：
γ = 1
β = 0
result5 = result4 * γ + β
print("[(input1-Ex)/sqrt(VarX+eps)] * γ + β:\n", result5, "\n") # 结果和图上一致


input1:
 tensor([[[[ 1.,  6.],
          [ 9.,  4.]],

         [[12., 18.],
          [13., 11.]]],


        [[[ 2.,  7.],
          [ 3.,  8.]],

         [[19., 17.],
          [15., 11.]]]]) 

Ex:
 tensor([ 9.2500, 10.2500]) 

VarX:
 tensor([25.9375, 35.1875]) 

input1-Ex:
 tensor([[[[-8.2500, -4.2500],
          [-0.2500, -6.2500]],

         [[ 2.7500,  7.7500],
          [ 3.7500,  0.7500]]],


        [[[-7.2500, -3.2500],
          [-6.2500, -2.2500]],

         [[ 9.7500,  6.7500],
          [ 5.7500,  0.7500]]]]) 

sqrt(VarX+eps):
 tensor([5.0929, 5.9319]) 

(input1-Ex)/sqrt(VarX+eps):
 tensor([[[[-1.6199, -0.7165],
          [-0.0491, -1.0536]],

         [[ 0.5400,  1.3065],
          [ 0.7363,  0.1264]]],


        [[[-1.4236, -0.5479],
          [-1.2272, -0.3793]],

         [[ 1.9144,  1.1379],
          [ 1.1290,  0.1264]]]]) 



# BatchNorm和LayerNorm的区别
BatchNorm是在批次上对所有样本的某一通道上做归一化。它假设样本在不同通道上的分量之间具备一定的独立性【一张RGB图像的每个通道（或者说维度）单独拎出来就是一张灰度图，不需要过多的借助其他维度就可以表达出图像的语义信息，换句话说，每个维度一定程度上都是相互独立的；原始的输入如此，卷积过程中的张量亦也，因为卷积结果的每一个通道都使用了不同的卷积核提取信息的结果，卷积核之间是没什么联系的，因此，卷积结果得每个通道一定程度上也可以认为是相互独立的】。它需要利用批量数据的统计信息。他需要维护全局的待学习参数（γ，β），供测试时使用。中文名称为批量归一化，其实就是想表达每个样本的每个通道都利用了该批次数据的统计信息进行了归一化。  

LayerNorm是在样本上对所有通道整体做一次归一化。它假设样本之间是具备一定的独立性（比起前面的通道分量上的独立假设，这显然是一个更弱的假设）。它仅需要利用样本本身，不需要利用批量数据的统计信息。中文名称为层归一化，其实就是想表达某个样本经过神经网络输出后，再经过LayerNorm处理，可以得到一个归一化结果。  

<p align="center">
<img src="./imgs/4类Norm对比.png"
    width="1000" /></p>


# CV较多使用BatchNorm而NLP较多使用LayerNorm

CV中的输入张量维度一般为(N,C,H,W)，H * W一般比较大（即，单通道的信息比较多），和所属batch中的其他样本的该通道一起进行批归一化，一方面可以使得数据规范化（可以有效地解决深度神经网络中的梯度消失问题），另一方面也间接做了数据增强（能够提高模型的泛化性能）。如果要是做LayerNorm的话，就没有用到第二个优点。

NLP中的输入张量维度一般为(L,N,E)，L为sequence length，N为batch size, E 为embedding dimension。
<p align="center">
<img src="./imgs/LayerNorm2.svg"
    width="700" /></p>

下面说一说(L,N,E)的张量不适合做BatchNorm的原因：
1. (L,N,E)的张量中可以把E看做通道数，那每一层就是一个值，这一点远小于图像中的长*宽，同时E一般也要比图像中的C大很多。那这样的话，如果再使用BatchNorm的话性价比就很低了（BN待学习的参数多了，纠正数据偏差的能力弱了）。
2. 输入的句子有长有短，如果做成batch后需要用0填充，这对于利用batch数据做数据偏差纠正也是一件麻烦事。  

所以，(L,N,E)的张量只能退而求其次，使用LayerNorm把每个时间步经过网络后的结果归一化一下也不错了。